In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.trend import AroonIndicator
from ta.trend import SMAIndicator
from ta.trend import DPOIndicator
from ta.trend import MACD
from ta.trend import EMAIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import OnBalanceVolumeIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
#from ta.others import DailyLogReturnIndicator
from ta.momentum import ROCIndicator

from IPython.display import clear_output
from datetime import datetime

start=(datetime.now())

payload=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].iloc[:,0])
payload=payload[payload!='BRK.B']
payload=payload[payload!='BF.B']
#print(payload)


class algo:
    start="2010-01-04"
    end='2021-12-04'
    interval='1mo'

    dff=pd.DataFrame()
    
    def __init__(self,stock):
        self.stock=stock
        
    # get raw stock data       
    def getdata(self):
        try:
            df=(yf.download(self.stock,algo.start,algo.end,interval=algo.interval).dropna())

            #df['log_current']=np.log(df.Close/df.Open)
            df['log_current']=((df.Close-df.Open)/df.Open)
            df['stock']=self.stock
            df['Year'] = df.index.strftime('%Y-%m-%d').str[:4]

            #output = stock data from start to end
            clear_output(wait=False)
        except:
            pass
    
        return(df)
    
    
    def stockloop():
        l1=0
        l2=500
        dff=algo.dff
        boom=pd.DataFrame()
        for i in range (l1,l2):
            try:
                rawdata=algo(payload[i]).getdata()
                dff=pd.concat([dff,rawdata])
                print(i)
            except Exception as e:
                print(e)
                pass
        dff.set_index([dff.stock,dff.index],inplace=True)
        return(dff)

dff=algo.stockloop()
#dff.to_csv('raw.csv')
uniq=(dff.index.unique(level=0))

finish = datetime.now()
print('Duration: {}'.format(finish - start))


499
Duration: 0:03:15.307918


In [19]:
start=(datetime.now())
#print(dff)
class algo2:
    dfmass=pd.DataFrame()
    thr1=1
    thr2=99
    end=['2022-01-01','2021-01-01','2020-01-01','2019-01-01','2018-01-01','2017-01-01','2016-01-01','2015-01-01','2014-01-01']
    def __init__(self,stock):
        self.stock=stock

    def indicators(self):
        df=self.stock
        #df['ema5']=EMAIndicator(close=df.Close.shift(1),window=5).ema_indicator()
        #df['ema10']=EMAIndicator(close=df.Close.shift(1),window=10).ema_indicator()
        #df['ema20']=EMAIndicator(close=df.Close.shift(1),window=20).ema_indicator()
        ##df['log_past']=DailyLogReturnIndicator(close=df.Close.shift(1)).daily_log_return()
        #df['ema1']=((df.Close-df.Close.shift(1))/df.Close.shift(1)).shift(1)
        #df['condition']=df.ema1
        #df['condition']=((df.Close-df.Close.shift(1))/df.Close.shift(1)).shift(1)
       # df['condition']=EMAIndicator(close=df.Close.shift(1),window=5).ema_indicator()
        #df['condition']=OnBalanceVolumeIndicator(close=df.Close.shift(1),volume=df.Volume.shift(1)).on_balance_volume()
        #df['condition']=ForceIndexIndicator(close=df.Close.shift(1),volume=df.Volume.shift(1),window=12).force_index()
        #df['condition']=AroonIndicator(close=df.Close.shift(1),window=12).aroon_indicator()
        df['condition']=MACD(close=df.Close.shift(1)).macd()
        #df['condition']=RSIIndicator(close=df.Close.shift(1),window=12).rsi()
        #df['condition']=ROCIndicator(close=df.Close.shift(1)).roc()
        df=df.dropna()

        #output = stock data with indicators
        return(df)

    def percentiles(self):
        df=self.stock
        dic1={}
        dic2={}
        for t in range (1,len(algo2.end)):
            try:
                time=algo2.end[t]
                df=(df[df.index<time])
  
                dic1[time]=np.nanpercentile((df.condition),algo2.thr1)
                dic2[time]=np.nanpercentile((df.condition),algo2.thr2)
                #print(dic1,dic2)
            except:
                pass
        #output = 2 dictionaries: year - percentile of preveious year (1,99)

        return(dic1,dic2)

    def trade(self,perc1,perc2):
        df=self.stock
        dff=pd.DataFrame()
        for t in range (1,len(algo2.end)):
            origin=algo2.end[t]
            end=algo2.end[t-1]
            #print(orgin,end)
            #select origin<stock data<end
            dft=(df[(df.index>=origin) & (df.index <end)])
            
            #apply trade conditions based on thresholds
            dft['trade_pass']=np.select([(dft.condition<perc1[origin]),(dft.condition>perc2[origin])],[1,-1],default=0)
            dft['Bottom_Percentile']=perc1[origin]
            dft['Top_Percentile']=perc2[origin]
            dff=pd.concat([dff,dft])
        return(dff)
    def stockloop():
        boom=pd.DataFrame()
        for i in uniq:

            rawdata=(dff.loc[i])
            ta=algo2(rawdata).indicators()
            #percentiles=algo2(ta).percentiles()
            trades=algo2(ta).trade(algo2(ta).percentiles()[0],algo2(ta).percentiles()[1])
            boom=pd.concat([boom,trades])

        return(boom)


boom=algo2.stockloop()
#print(boom)
boom.to_csv('all_data.csv')

finish = datetime.now()
print('Duration: {}'.format(finish - start))


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Duration: 0:00:32.825725


In [20]:
def yearloop(boom):

    years=np.array(['2021','2020','2019','2018','2017','2016','2015','2014'])
   
    long=boom[boom['trade_pass']==1]
    short=boom[boom['trade_pass']==-1]        
    short['log_current']=-short['log_current']+1
    long['log_current']=long['log_current']+1
    group=([long,short])
    long.to_csv('all_long.csv')
    short.to_csv('all_short.csv')

    #print(short)
    temp=np.array([])
    for g in group: 
     
        for y in years:

            gr=g[g.Year==y]
    
            trade_count=gr.shape[0]
            try:
                win_rate=(((gr[gr['log_current']>1]).shape[0]))/trade_count
            except:
                win_rate=0
            cumilative=np.prod(gr.groupby(level=0).mean().log_current)-1
            minimum=np.min(gr.groupby(level=0).mean().log_current)-1
            temp1=np.array([y,trade_count,win_rate,cumilative,minimum])
            temp=np.append(temp,temp1)

    return(temp)

daf=(yearloop(boom).reshape(16,5))
dper=pd.DataFrame(daf)
dper['stock_count']=np.count_nonzero(uniq)
print(dper.round(1))
dper.to_csv('donut.csv')

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


       0     1                    2                     3  \
0   2021    88   0.6704545454545454    0.0906566440080907   
1   2020  1223   0.6304170073589533  0.009051701399766188   
2   2019   734   0.6444141689373297   0.36933064892174783   
3   2018   580   0.5137931034482759  -0.10255861918363385   
4   2017   482   0.5912863070539419    0.1454223924511664   
5   2016  1221   0.6445536445536445    0.3308963665981126   
6   2015   715   0.4461538461538462  -0.12275869135245832   
7   2014   450   0.6955555555555556    0.2844469585350162   
8   2021  2148   0.4227188081936685  -0.14953990151620222   
9   2020   944   0.4851694915254237   -0.3127665205714899   
10  2019   967    0.390899689762151  -0.20668631901016266   
11  2018  1937   0.5131646876613319  0.029500471325453415   
12  2017  1543   0.3668178872326636  -0.19773106215633396   
13  2016   863  0.46929316338354576   -0.1009314169272375   
14  2015  1835   0.5095367847411444  -0.03993204075898238   
15  2014  3133   0.40663